# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


In [1]:
import pandas as pd
import numpy as np

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/ealaxi/paysim1 . Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [4]:
# Your code here

data = pd.read_csv('../PS_20174392719_1491204439457_log.csv')
data = data.sample(100000)
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1485231,141,CASH_IN,59799.70,C159618322,6778417.42,6838217.12,C769244897,186159.74,126360.04,0,0
5985982,408,CASH_OUT,223999.83,C1326737897,22080.00,0.00,C820908785,729046.87,953046.70,0,0
2254415,187,CASH_IN,73486.86,C153957517,23693557.23,23767044.10,C1570556626,3515296.62,3441809.76,0,0
6183784,567,PAYMENT,3901.59,C433332833,0.00,0.00,M1376663395,0.00,0.00,0,0
6080740,520,CASH_IN,418247.50,C124243050,3745436.53,4163684.04,C1967864415,989150.23,570902.72,0,0


In [5]:
data.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [6]:
data.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,100000.000000,100000.0
mean,243.391280,1.803313e+05,8.281403e+05,8.488994e+05,1.107356e+06,1.231279e+06,0.001350,0.0
std,142.233392,6.223255e+05,2.892470e+06,2.926579e+06,3.462346e+06,3.730723e+06,0.036718,0.0
min,1.000000,8.700000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,156.000000,1.354550e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,239.000000,7.525446e+04,1.432000e+04,0.000000e+00,1.325252e+05,2.162646e+05,0.000000,0.0
75%,334.000000,2.097559e+05,1.074115e+05,1.451257e+05,9.396259e+05,1.111239e+06,0.000000,0.0
max,730.000000,4.710786e+07,3.844183e+07,3.856340e+07,2.511150e+08,2.506381e+08,1.000000,0.0


In [ ]:
#Maybe Type and Amount could be important features to determine fraud. 

### What is the distribution of the outcome? 

In [8]:
# Your response here
print(data['isFraud'].value_counts())
print(data['isFlaggedFraud'].value_counts())

0    99865
1      135
Name: isFraud, dtype: int64
0    100000
Name: isFlaggedFraud, dtype: int64


### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [12]:
# Your code here
display(data.isnull().sum())
display(data['step'].value_counts())
display(data['type'].value_counts())


step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

18     825
19     791
187    779
307    771
235    736
      ... 
459      1
616      1
84       1
726      1
658      1
Name: step, Length: 469, dtype: int64

CASH_OUT    35306
PAYMENT     33583
CASH_IN     22047
TRANSFER     8412
DEBIT         652
Name: type, dtype: int64

### Run a logisitc regression classifier and evaluate its accuracy.

In [13]:
# Your code here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = data.drop(['step','type','nameOrig','nameDest','isFraud','isFlaggedFraud'],axis = 1) #features
y = data['isFraud'] #labels
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.99476

### Now pick a model of your choice and evaluate its accuracy.

In [15]:
# Your code here
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [16]:
model_2 = KNeighborsClassifier(n_neighbors = 4)
model_2 = model_2.fit(X_train, y_train)
model_2.predict(X_test)
np.array(y_test)
accuracy_score(model_2.predict(X_test),np.array(y_test))

0.99904

### Which model worked better and how do you know?

In [2]:
# Your response here
#I think both are working really well, but the last one is slightly better.
#Now we can code the confusion_matrix to see who is better.

In [19]:
from sklearn.metrics import confusion_matrix
pred_1 = model.predict(X_test)
pred_2 = model_2.predict(X_test)
display(confusion_matrix(y_test,pred_1))
display(confusion_matrix(y_test,pred_2))

array([[24843,   118],
       [   13,    26]], dtype=int64)

array([[24961,     0],
       [   24,    15]], dtype=int64)

In [ ]:
#This shows that the best results are given by the logistic regression.
#Because LR is better detecting fraud than KNN as we can see in the confusion_matrix.



### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.